In [2]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg

In [4]:
adata = sc.read_10x_h5(
    filename="../data/slide-5434/cell_feature_matrix.h5"
)
df = pd.read_csv("../data/slide-5434/cells.csv.gz")
adata
df

,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area
0,aaaabpli-1,1675.609375,5619.336426,466,0,0,1,4,471,732.931120,75.817347
1,aaaadblk-1,1659.425903,5539.659668,1300,1,0,6,56,1363,1062.255664,103.272347
2,aaabbfid-1,1663.462036,5458.263184,469,0,0,1,10,480,713.242995,15.127344
3,aaabkhnj-1,1652.523682,5481.042969,362,0,0,1,3,366,521.148300,38.473126
4,aaabmfhp-1,1643.152710,5502.311035,522,0,0,2,12,536,627.942835,29.035470
...,...,...,...,...,...,...,...,...,...,...,...
160990,oinkpcol-1,9505.174805,17938.175781,542,0,0,4,20,566,439.009078,47.955939
160991,oinlgpih-1,9437.881836,17942.253906,734,0,0,0,7,741,803.194248,69.901878
160992,oinlofdk-1,9439.881836,17967.841797,325,0,0,2,5,332,679.375806,41.769533
160993,oinlpiem-1,9458.916016,18018.853516,660,0,0,6,16,682,932.702378,16.436876


In [6]:
df.set_index(adata.obs_names, inplace=True)
adata.obs = df.copy()
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()
adata.obs

,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area
aaaabpli-1,aaaabpli-1,1675.609375,5619.336426,466,0,0,1,4,471,732.931120,75.817347
aaaadblk-1,aaaadblk-1,1659.425903,5539.659668,1300,1,0,6,56,1363,1062.255664,103.272347
aaabbfid-1,aaabbfid-1,1663.462036,5458.263184,469,0,0,1,10,480,713.242995,15.127344
aaabkhnj-1,aaabkhnj-1,1652.523682,5481.042969,362,0,0,1,3,366,521.148300,38.473126
aaabmfhp-1,aaabmfhp-1,1643.152710,5502.311035,522,0,0,2,12,536,627.942835,29.035470
...,...,...,...,...,...,...,...,...,...,...,...
oinkpcol-1,oinkpcol-1,9505.174805,17938.175781,542,0,0,4,20,566,439.009078,47.955939
oinlgpih-1,oinlgpih-1,9437.881836,17942.253906,734,0,0,0,7,741,803.194248,69.901878
oinlofdk-1,oinlofdk-1,9439.881836,17967.841797,325,0,0,2,5,332,679.375806,41.769533
oinlpiem-1,oinlpiem-1,9458.916016,18018.853516,660,0,0,6,16,682,932.702378,16.436876


Subset the cells in the entire slide to the three separate tissues: